In [ ]:
import tensorflow as tf
from tensorflow_probability import distributions as tfd
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import helper
import models

import matplotlib as mpl
import pickle
from copy import deepcopy
import time
#from graphviz import Digraph
import itertools
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
import seaborn as sns
from scipy.stats import multivariate_normal

In [ ]:
SIGMA_R = .001
SIGMA_R_MODEL = .1

Tx = 100
Ty = Tx

Z_PRIOR = "uniform" #"uniform" / "informative" / "normal"
SIGMA_0_PRIOR = np.array([[1., 0.], [0., 1.]])

#pnas: diagonal [north: -45, south: 45]
#pnas: cardinal [north: 90, south: 0]
#ground-truth-shared: cardinal [north: 180, south: 0]
NORTH_ALPHA = 90
SOUTH_ALPHA = 0

NUM_OF_DATASETS = 20


In [ ]:
datasets = []
mllh_1x2D = []
mllh_2x2Ds = []

for i in range(NUM_OF_DATASETS):
  data1 = helper.generate_data(Tx, alpha=NORTH_ALPHA, context_value=0, z_prior_type=Z_PRIOR, sigma_reward=SIGMA_R)
  data2 = helper.generate_data(Ty, alpha=SOUTH_ALPHA, context_value=1, z_prior_type=Z_PRIOR, sigma_reward=SIGMA_R)
  
  true_data = helper.concatenate_data(data1, data2)
  interleaved_indices = helper.riffle(np.arange(Tx+Ty)[:Tx],np.arange(Tx+Ty)[Tx:])
  true_data = helper.reorder_data(true_data,interleaved_indices)
  
  mllh_1x2D.append(models.mllh_analytic_1x2D(true_data, SIGMA_R_MODEL, Sigma_0 = SIGMA_0_PRIOR))
  mllh_2x2Ds.append(models.mllh_analytic_2x2D_shared(true_data, SIGMA_R_MODEL, Sigma_0 = SIGMA_0_PRIOR))
  datasets.append(true_data)

In [ ]:
plt.plot([mllh_1x2D[0],mllh_2x2Ds[0]])
plt.yscale('log')


In [ ]:
helper.plot_data(datasets[0], limit=1)

In [ ]:
plt.plot([mllh_1x2D,mllh_2x2Ds])
plt.yscale('log')
plt.xlabel('0 = 1x2D, 1 = 2x2D-shared')
plt.ylabel('log mllh')
plt.show()

In [ ]:
plt.plot(np.array([mllh_1x2D,mllh_2x2Ds]).T)
plt.yscale('log')
plt.xlabel('0 = 1x2D, 1 = 2x2D-shared')
plt.ylabel('log mllh')
plt.show()

In [ ]:
samples = models.mllh_analytic_2x2D_shared(true_data, SIGMA_R_MODEL, Sigma_0 = SIGMA_0_PRIOR, return_posterior=True)['mu_is']

In [ ]:
ax = plt.axes()
[ax.arrow(0, 0, sample[0], sample[1], head_width=0.02, head_length=0.05, fc='k', ec='k') for sample in samples]
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
helper.plot_data(true_data, limit=1)

In [ ]:
ax = plt.axes()
ax.arrow(0, 0, samples[-1][0], samples[-1][1], head_width=0.02, head_length=0.05, fc='k', ec='k') 
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
plt.plot(np.array(samples))
plt.gca().legend(('gamma_1','gamma_2'))
plt.show()